## Helpful links
* Great overview of wavetable oscillator design:  https://www.earlevel.com/main/category/digital-audio/oscillators/wavetable-oscillators/
* Formulas for converting notes to frequency:  http://pages.mtu.edu/~suits/NoteFreqCalcs.html
* A chart of piano key frequencies:  https://en.wikipedia.org/wiki/Piano_key_frequencies


## Wavetable design
### Table size
When determining table size, it's important to consider how many harmonics will fit in the table.  For example:

1.  Consider a table size of 2048 samples.
2.  The max harmonic that can be supported by that table is 1024.  That would be a sinusoid that cycles every other sample - which is the fastest we can represent due to Nyquist.  So, a 2048 sample single cycle wave (sampled at 44100 Hz) would be 21.5332031 Hz (44100 cycles/second / 2048 samples/cycle).
3.  To find the closest note to render, find the closest note on the piano key frequencies chart - in this case F0 and then figure out the number of cents to detune it by to get a single 2048 samples wave.  
4.  Using the equation from the [NoteFreqCalcs website](http://pages.mtu.edu/~suits/NoteFreqCalcs.html), we see that we need F0 tuned to -23 cents to hit an exact 2048 sample single cycle at 44100 Hz.
5.  In that F0 -22 note sampled at 44100 Hz, our highest harmonic is 22050 (21.5332031 * 1024) which is the highest frequency we can get without aliasing.

### Number of tables
We need to have more than 2048 sample waveform describe the entire frequency range of the wavetable.  This is because as we increase in frequency and scan through the table, our higher overtones will begin to alias.  For example, if we go up 1 octave to F1, which is at 43.65353 Hz (twice F0 as it is an octave up), then we will start aliasing after 505 overtones and we alias all the way down to 600 Hz which would be very audible.  The question is how much aliasing can we allow before it becomes detrimental to the sound?

Consider that most people hear sound to about 15kHz so we want the aliasing to become inaudible by the time it reflects back to 15kHz.  That means we can allow harmonics up to about (44.1k - 15k =) 29.1kHz.  29.1k / 22050 = 1.319 ratio of the non-aliased max frequency.  Let's call it 1.3333 or 4/3 * non-aliased max frequency.  This means our naive  max frequency calculation of 1 / (2 * maxHarmonic) becomes (4/3) * (1/(2 * maxHarmonic)) which reduces to 2 / (3 * maxHarmonic) or even better: 2 / 3 / maxHarmonic.  Note - these calculations for max frequency are normalized to the sample rate - so in our case multiply them by 44100 Hz.  

So to determine the number of tables (and what frequency ranges they represent)
1.  Calculate the max frequency for the number of harmonics in the current table
2.  In the next table, reduce the number of harmonics by 1/2 (up an octave).
3.  Do this until you only have 1 harmonic in your table.

Here is an example (all of these alias back down to about 14700 Hz):

 Table | # of Harmonics | Max Non-aliasing Frequency | Top Frequency 
 ----- | :--------------: | :--------------------------: | :-------------: 
 1 | 1024 | 21.53 Hz | 28.71 Hz 
 2 | 512 | 43.06 Hz | 57.42 Hz 
 3 | 256 | 86.13 Hz | 114.84 Hz 
 4 | 128 | 172.26 Hz | 229.68 Hz 
 5 | 64 | 344.53 Hz | 459.37 Hz 

 and so on...

